In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from preprocessing_utils import process_sat_data

In [3]:
import visdom

In [4]:
vis = visdom.Visdom()

Setting up a new session...


In [5]:
data_train = pd.read_csv('../data/train.csv')
data_test = pd.read_csv('../data/track1/test.csv')

In [6]:
train_data = data_train.copy(deep=True)
test_data = data_test.copy(deep=True)

In [7]:
process_sat_data(train_data, test_data, scale=10000)

In [8]:
def plot_axis(sat_id, axis, ellipse_index, span=1):
    data = train_data[train_data['sat_id'] == sat_id]
    label = data['cluster']
    time = data['epoch']
    data = data.loc[:, ['x_sim', 'y_sim', 'z_sim']].values
    vis.line(data[ellipse_index * 24: (ellipse_index + span) * 24, axis], time[ellipse_index * 24: (ellipse_index + span) * 24], opts={'markersize': 4}, win='Axis'+str(axis))

In [9]:
sat_id = 1
plot_axis(sat_id, 0, 0, 11)
plot_axis(sat_id, 1, 0, 11)
plot_axis(sat_id, 2, 0, 11)

In [10]:
import torch
import torch.nn as nn

In [11]:
from torch.utils.data import DataLoader
import torch.optim as optim

In [12]:
class SatNet(nn.Module):
    def __init__(self, name, omega, mean, diff):
        super(SatNet, self).__init__()
        self.name = name
        self.w = nn.Parameter(omega)
        self.off = nn.Parameter(mean)
        self.a1 = nn.Parameter(diff)
        self.a2 = nn.Parameter(diff)
        self.p1 = nn.Parameter(torch.randn(1))
        self.p2 = nn.Parameter(torch.randn(1))
    def forward(self, t):
        s1 = self.a1 * torch.cos(self.p1 + self.w * t)
        s2 = self.a2 * torch.cos(self.p2 + self.w * t * 2)
        s = self.off + s1 + s2
        v1 = - self.a1 * self.w * torch.sin(self.w * t + self.p1)
        v2 = - self.a2 * self.w * torch.sin(self.w * t + self.p2) * 2
        v = v1 + v2
        return [s, v]

In [13]:
def smape(satellite_predicted_values, satellite_true_values):  
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [14]:
def get_initial_values(ellipse_data):
    omega = 2 * np.pi / (torch.max(ellipse_data[:, 6]) - torch.min(ellipse_data[:, 6])) # wT = 2(pi)
    mean = torch.mean(ellipse_data[:,[0,1,2]], dim=0)
    amplitude_x = 0.5 * (torch.max(ellipse_data[:,0]) - torch.min(ellipse_data[:,0]))
    amplitude_y = 0.5 * (torch.max(ellipse_data[:,1]) - torch.min(ellipse_data[:,1]))
    amplitude_z = 0.5 * (torch.max(ellipse_data[:,2]) - torch.min(ellipse_data[:,2]))
    return [omega, mean, amplitude_x, amplitude_y, amplitude_z]

In [15]:
def print_status(itr, ellipse_data, loss_x, loss_y, loss_z,smape_x, smape_y, smape_z):
    print('Iteration: ', itr)
    print('Loss: {} (X) | {} (Y) | {} (Z)'.format(loss_x.item(), loss_y.item(), loss_z.item()))
    print('SMAPE: {} (X) | {} (Y) | {} (Z)'.format(smape_x, smape_y, smape_z))

In [16]:
df_cols = ['sat_id', 'ellipse_id', 'smape_x', 'smape_y', 'smape_z',
           'a1_x', 'a2_x', 'p1_x', 'p2_x', 'w_x', 'off_x',
           'a1_y', 'a2_y', 'p1_y', 'p2_y', 'w_y', 'off_y',
           'a1_z', 'a2_z', 'p1_z', 'p2_z', 'w_z', 'off_z']

In [17]:
def save_weights(sat_nets, sat_id, ellipse_id, smape_x, smape_y, smape_z, weights):
    row = [sat_id, ellipse_id, smape_x, smape_y, smape_z]
    for axis in ['x', 'y', 'z']:
        model = sat_nets[sat_id][ellipse_id][axis]
        params = [model.a1, model.a2, model.p1, model.p2, model.w, model.off]
        params = [item.item() for item in params]
        row += params
    weights = np.concatenate([weights.values, [row]], axis=0)
    weights = pd.DataFrame(weights, columns=df_cols)
    weights.to_excel('../data/weights.xlsx', index=False)
    return weights

In [20]:
def train(train_data):
    sat_nets = {}
    weights = pd.DataFrame([[-1] * len(df_cols)], columns=df_cols)
    weights.to_excel('../data/weights.xlsx', index=False)
    for sat_id in range(1):
        
        print('------- Satellite ID:', sat_id, '------- ')
        cols = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim', 'epoch']
        sat_data = train_data[train_data['sat_id'] == sat_id].loc[:, cols].values
        print('\n## Number of Ellipses: ', int(sat_data.shape[0]/24)+1, '\n')
        sat_nets[sat_id] = {}
        data = DataLoader(sat_data, batch_size=24) # 24 points per ellipse
        data_iters = iter(data)
        
        for ellipse_id, ellipse_data in enumerate(data_iters):
            
            print('\n**** Ellipse ID:', ellipse_id, '****\n')
            ellipse_data = ellipse_data.float()
            omega, mean, amplitude_x, amplitude_y, amplitude_z = get_initial_values(ellipse_data)
            sat_nets[sat_id][ellipse_id] = {
                'x': SatNet('x', omega, mean[0], amplitude_x),
                'y': SatNet('y', omega, mean[1], amplitude_y),
                'z': SatNet('z', omega, mean[2], amplitude_z)
            }
            
            optim_x = optim.Adam(sat_nets[sat_id][ellipse_id]['x'].parameters(), 0.05, (0.9, 0.999))
            optim_y = optim.Adam(sat_nets[sat_id][ellipse_id]['y'].parameters(), 0.05, (0.9, 0.999))
            optim_z = optim.Adam(sat_nets[sat_id][ellipse_id]['z'].parameters(), 0.05, (0.9, 0.999))
            
            for itr in range(1):
                optim_x.zero_grad()
                optim_y.zero_grad()
                optim_z.zero_grad()
                
                time = ellipse_data[:,6].float()
                x_pred, vx_pred = sat_nets[sat_id][ellipse_id]['x'](time)
                y_pred, vy_pred = sat_nets[sat_id][ellipse_id]['y'](time)
                z_pred, vz_pred = sat_nets[sat_id][ellipse_id]['z'](time)
                
                loss_sx = torch.sqrt(torch.mean((x_pred - ellipse_data[:,0]) ** 2))
                loss_sy = torch.sqrt(torch.mean((y_pred - ellipse_data[:,1]) ** 2))
                loss_sz = torch.sqrt(torch.mean((z_pred - ellipse_data[:,2]) ** 2))
                loss_vx = torch.sqrt(torch.mean((vx_pred - ellipse_data[:,3]) ** 2))
                loss_vy = torch.sqrt(torch.mean((vy_pred - ellipse_data[:,4]) ** 2))
                loss_vz = torch.sqrt(torch.mean((vz_pred - ellipse_data[:,5]) ** 2))
                loss_x = loss_sx + loss_vx
                loss_y = loss_sy + loss_vy
                loss_z = loss_sz + loss_vz
                
                loss_x.backward()
                loss_y.backward()
                loss_z.backward()
                
                optim_x.step()
                optim_y.step()
                optim_z.step()
                
                if(itr % 1000 == 0):
                    smape_x = smape(x_pred.detach().numpy(), ellipse_data[:,0].numpy())
                    smape_y = smape(y_pred.detach().numpy(), ellipse_data[:,1].numpy())
                    smape_z = smape(z_pred.detach().numpy(), ellipse_data[:,2].numpy())
                    print_status(itr, ellipse_data, loss_x, loss_y, loss_z, smape_x, smape_y, smape_z)
                    
            weights = save_weights(sat_nets, sat_id, ellipse_id, smape_x, smape_y, smape_z, weights)

In [21]:
sat_nets = train(train_data)

------- Satellite ID: 0 ------- 

## Number of Ellipses:  40 


**** Ellipse ID: 0 ****

Iteration:  0
Loss: 5.576506614685059 (X) | 7.638147830963135 (Y) | 13.015547752380371 (Z)
SMAPE: 0.7003733515739441 (X) | 0.5923134684562683 (Y) | 0.881243884563446 (Z)

**** Ellipse ID: 1 ****

Iteration:  0
Loss: 2.929800510406494 (X) | 6.373621940612793 (Y) | 9.448028564453125 (Z)
SMAPE: 0.65348881483078 (X) | 0.6465634107589722 (Y) | 0.5417823791503906 (Z)

**** Ellipse ID: 2 ****

Iteration:  0
Loss: 5.160072326660156 (X) | 6.962034702301025 (Y) | 9.618172645568848 (Z)
SMAPE: 0.7201976776123047 (X) | 0.49835148453712463 (Y) | 0.7670266032218933 (Z)

**** Ellipse ID: 3 ****

Iteration:  0
Loss: 4.24979305267334 (X) | 8.606908798217773 (Y) | 9.263513565063477 (Z)
SMAPE: 0.5899756550788879 (X) | 0.5822843909263611 (Y) | 0.5310347676277161 (Z)

**** Ellipse ID: 4 ****

Iteration:  0
Loss: 3.7636265754699707 (X) | 6.447924613952637 (Y) | 11.566582679748535 (Z)
SMAPE: 0.4715949296951294 (X) | 0.463